# MLOPs Zoomcamp 2024 / 01-homework
By Valerie G., May 2024

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [2]:
def get_samples(df_cp, dataset_type, dv=DictVectorizer()):
    df = df_cp.copy()  # to avoid warning about changing original df
    print('dataset_type:', dataset_type)
    num_columns = len(df.columns)  # for homework answer
    print('num_columns:', num_columns)
    print('num_rows:', len(df))
    
    # calc duration and add to df
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)  # convert to min
    
    # for homework answers
    if dataset_type == 'train':
        duration_std = df.duration.std()
        percentage_not_outliers = ((df.duration >= 1) & (df.duration <= 60)).mean() * 100

    # remove outliers
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    # pic feature columns and convert them to dictionaries
    location_colns = ['PULocationID', 'DOLocationID']
    df[location_colns] = df[location_colns].astype(str)
    input_dicts = df[location_colns].to_dict(orient='records')
    
    # convert dict to input features for the model
    if dataset_type == 'train':
        inputs = dv.fit_transform(input_dicts)
    else:
        inputs = dv.transform(input_dicts)
    print('inputs.shape', inputs.shape)
        
    # pic target column and get its values
    target_coln = 'duration'
    targets = df[target_coln].values
    print('targets.shape', targets.shape)
    
    # print homework answers
    if dataset_type == 'train': 
        print('\nQ1:', num_columns)
        print('Q2:', duration_std)
        print('Q3:', percentage_not_outliers)
        matrix_dim = inputs.shape[1]
        print('Q4:', matrix_dim, '\n')
    
    return inputs, targets, dv


def evaluate_model(model, inputs, targets):
    preds = model.predict(inputs)  # predict
    rmse = root_mean_squared_error(targets, preds)  # evaluate
    return rmse

In [3]:
# read data
df_train = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')
df_val = pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')

# show data
df_train

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


In [4]:
# load dataset for model
X_train, y_train, dv = get_samples(df_train, dataset_type='train')
X_val, y_val, _ = get_samples(df_val, dataset_type='val', dv=dv)

dataset_type: train
num_columns: 19
num_rows: 3066766
inputs.shape (3009173, 515)
targets.shape (3009173,)

Q1: 19
Q2: 42.59435124195458
Q3: 98.1220282212598
Q4: 515 

dataset_type: val
num_columns: 19
num_rows: 2913955
inputs.shape (2855951, 515)
targets.shape (2855951,)


Output:
```
dataset_type: train
num_columns: 19
num_rows: 3066766
inputs.shape (3009173, 515)
targets.shape (3009173,)

Q1: 19
Q2: 42.59435124195458
Q3: 98.1220282212598
Q4: 515 

dataset_type: val
num_columns: 19
num_rows: 2913955
inputs.shape (2855951, 515)
targets.shape (2855951,)
```

In [5]:
# train model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [6]:
# evaluate
rmse_train = evaluate_model(model, X_train, y_train)  # on train set
print('Q5:', rmse_train)
rmse_val = evaluate_model(model, X_val, y_val)  # on validation set
print('Q6:', rmse_val)

Q5: 7.649261934850555
Q6: 7.811817745843695


Output:
```
Q5: 7.649261934850555
Q6: 7.811817745843695
```